In [7]:
import pandas as pd

dta_security=pd.read_csv("https://raw.githubusercontent.com/ucr-courses/knn-lab/main/data/boston_housing.csv")
dta_security.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


Let's keep 10% of data for testing and remaning for training/validation

In [ ]:
from sklearn.model_selection import train_test_split
[X_train_val, X_test, y_train_val, y_test] = train_test_split(dta_security.drop('Threat',axis=1), ???, test_size=???, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_scaler=scaler.fit(???)
X_train_val_scaled=X_scaler.transform(???)
X_test_scaled=X_scaler.transform(???)

Setup up KNN model in scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model= KNeighborsClassifier()

Because we are interested in evaluating a variety of different possible values of $k$ in our model, we create a dictionary of values to explore using the label *n_neighbors* and a **range** of values between 1 and 10 (with the latter value being set to 10 + 1 since the end of **range** is exclusive). Enter:

In [ ]:
paramGrid={'n_neighbors' : range( 1 , 10 + 1 )}

Now we are ready to fit our models and evaluate the different $k$ values. To do this, we will use $k$-fold cross-validation, where the $k$ in $k$-fold vs. the $k$ in KNN are not to be confused. Before we get started, we need to set up the procedure using the scikit-learn **GridSearchCV** function and store it in a variable with the label search. Within the function, the first two arguments are used to specify our model and our paramGrid from above; in addition, we use *cv* to set the number of folds to 10 and choose `'accuracy'` as our *scoring* metric to have results comparable to what is shown in the text (alternative scoring metrics may also be evaluated, which can be useful in various cases such as high class imbalance). Enter:

In [ ]:
from sklearn.model_selection import GridSearchCV

search=GridSearchCV(???)

Now execute cross_validation

In [ ]:
searchFit=search.fit(???,???)

Check results

In [ ]:
pd.DataFrame(searchFit.cv_results_)[['mean_test_score','params','rank_test_score']].sort_values(by='rank_test_score')

,mean_test_score,params,rank_test_score
5,0.655556,{'n_neighbors': 6},1
8,0.648148,{'n_neighbors': 9},2
9,0.640741,{'n_neighbors': 10},3
6,0.622222,{'n_neighbors': 7},4
4,0.622222,{'n_neighbors': 5},5
3,0.618519,{'n_neighbors': 4},6
2,0.607407,{'n_neighbors': 3},7
7,0.603704,{'n_neighbors': 8},8
0,0.577778,{'n_neighbors': 1},9
1,0.570370,{'n_neighbors': 2},10


The best performing k is ???

In [ ]:
best_estimator=searchFit.best_estimator_
best_estimator

We can either use searchFit to make predictions or use best_estimator to make predictions. searchFit uses the best estimator by default. Therefore, the results will be the same.

In [ ]:
predicted_prob=searchFit.predict_proba(X_test_scaled)

In [ ]:
predicted_prob=best_estimator.predict_proba(X_test_scaled)
predicted_prob

array([[0.66666667, 0.33333333],
       [0.16666667, 0.83333333],
       [0.33333333, 0.66666667],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [0.        , 1.        ],
       [0.16666667, 0.83333333],
       [0.5       , 0.5       ],
       [0.        , 1.        ],
       [0.83333333, 0.16666667],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.83333333, 0.16666667],
       [0.5       , 0.5       ],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.66666667, 0.33333333],
       [0.16666667, 0.83333333],
       [0.33333333, 0.66666667],
       [0.33333333, 0.66666667],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.5       , 0.5       ]])

The first column in the output above shows the probabilities of the cases belonging to Class 0 (Not threat), while the second column lists the probabilities of the cases belonging to Class 1 (threat).

To classify these probability predictions, let's use a threshold probability that reflects the proportion of 1s in our train-validation dataset. NOte that in python index starts from 0, therefore predicted_prob[:,1] gives us the **second** column.

In [ ]:
cutoff_prob=???

In [ ]:
predicted_test=np.where(predicted_prob[:,1]>???,1,0)
predicted_test

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1])

In [ ]:
from sklearn.metrics import confusion_matrix
# first argument is true values, second argument is predicted values
confusion_matrix(???,???)

array([[ 7,  5],
       [ 2, 16]], dtype=int64)

The confusion matrix is arranged as follows

|                     | Predicted Negative | Predicted Positive |
|---------------------|--------------------|--------------------|
| **Actual Negative** | True Negatives (TN) | False Positives (FP) |
| **Actual Positive** | False Negatives (FN) | True Positives (TP) |


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(???,???)

0.7666666666666667

Alternatively we can use classification report for several other metrics

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predicted_test,target_names = ['Not threat', 'threat']))

              precision    recall  f1-score   support

  Not threat       0.78      0.58      0.67        12
      threat       0.76      0.89      0.82        18

    accuracy                           0.77        30
   macro avg       0.77      0.74      0.74        30
weighted avg       0.77      0.77      0.76        30

